<a href="https://colab.research.google.com/github/I3ryI3e/fantastic-octo-barnacle/blob/master/RNNCrypto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests        # for making http requests to binance
import json            # for parsing what binance sends back to us
import pandas as pd    # for storing and manipulating the data we get back
import numpy as np     # numerical python
from sklearn import preprocessing  #helps in the preprocessing function
from collections import deque
import matplotlib.pyplot as plt # for charts and such
import random
    
import datetime as dt  # for dealing with times

#Fetch data from how many days ago?
DAYS_AGO = 4
#1 Day = 86400000 miliseconds
DAY_TO_MILISECONDS = 86400000
#Days to miliseconds
DAYS_IN_MILISECONDS = DAYS_AGO * DAY_TO_MILISECONDS
#Number of samples (Max 1000)
LIMIT = 1000
#1 Minute = 60000 mili
MIN_EQUALS_MILS= 60000
# SEQ_LEN = How much hours are we going to give to the NN so it can predict 
SEQ_LEN = 60
# What Pair are we going to predict
RATIO_TO_PREDICT = 'ETHUSDT'
#How many hours ahead are we going to predict(if '1' we are going to predict if the price is going to be higher in the next hour)
FUTURE_PERIOD_PREDICT = 3
#Validation sample split
VALIDATION_PERCENTAGE = 0.05
TRAINING_DATA = 1-VALIDATION_PERCENTAGE


In [0]:
#Connect to binance API and get the data in each pair we want information on
# INPUTS: symbol = The pair(ex= LTC-USDT) // Interval = 1h(1 hour intervals)
# OUTPUTS: DataFrame with the data 

def get_bars(symbol, startTime ,interval = '1m'):
 root_url = 'https://api.binance.com/api/v1/klines'
 url = root_url + '?symbol=' + symbol + '&interval=' + interval + '&startTime=' + str(startTime) + '&limit=' + str(LIMIT) 
 data = json.loads(requests.get(url).text)
 df = pd.DataFrame(data)
 df.columns = ['open_time',
               'open_price', 'high', 'low', 'close_price', 'volume',
               'close_time', 'qav', 'num_trades',
               'taker_base_vol', 'taker_quote_vol', 'ignore']
 df=df.drop(columns=['close_time', 'qav', 'num_trades',
               'taker_base_vol', 'taker_quote_vol', 'ignore'])
 return df

In [0]:
#Does the targets for the network.
#INPUTS: Current price; Future Price
#OUTPUT: 1 if the Future Price is higher than the current price, 0 otherwise

def classify(current, future):
  if float(future) > float(current):
    return 1
  else:
    return 0

In [0]:
def preprocess(df):
  df = df.drop('future', 1)
  for col in df.columns:
    if col != 'target':
      df[col]=df[col].astype(float).pct_change()
      df.dropna(inplace=True)
      df[col] = preprocessing.scale(df[col].values)
      
  df.dropna(inplace=True)
  print(df.columns)
  sequential_data = []
  prev_days= deque(maxlen=SEQ_LEN)
  #Making the sequeces // appending values until we get to SEQ_LEN and add it to sequential_data
  #that's going to be a list of sequences
  for i in df.values:
      print(i)
      prev_days.append([n for n in i[:-1]])
      if len(prev_days) == SEQ_LEN:
        sequential_data.append([np.array(prev_days),i[-1]])      
  random.shuffle(sequential_data)

In [7]:
#Preparing all the DataFrame to work with

root_url = 'https://api.binance.com/api/v1/time'
now = json.loads(requests.get(root_url).text)
now = now.get('serverTime')
earlier5days = now - DAYS_IN_MILISECONDS
btcusdt = pd.DataFrame()
ethusdt = pd.DataFrame()
ltcusdt = pd.DataFrame()
xrpusdt = pd.DataFrame()

for x in range(earlier5days, now, (LIMIT*MIN_EQUALS_MILS)):
  print(x)
  btcusdt = pd.concat([btcusdt,get_bars('BTCUSDT',x)])
  ethusdt = pd.concat([ethusdt,get_bars('ETHUSDT',x)])
  ltcusdt = pd.concat([ltcusdt,get_bars('LTCUSDT',x)])
  xrpusdt = pd.concat([xrpusdt,get_bars('XRPUSDT',x)])

main_df = pd.DataFrame(ethusdt)
print(len(main_df))
main_df= pd.merge(main_df,btcusdt,on='open_time',how='left',suffixes=('_ETHUSDT','_BTCUSDT')).fillna(method='ffill')

ltcusdtColumNames=ltcusdt.columns.tolist()
renamedNamesLTC=[]
for name in ltcusdtColumNames:
  renamedNamesLTC.append(name+"_LTCUSDT")
      
renamedNamesLTC=dict(zip(ltcusdtColumNames, renamedNamesLTC))
ltcusdt=ltcusdt.rename(index=str, columns=renamedNamesLTC)

xrpusdtColumNames=xrpusdt.columns.tolist()
renamedNamesXRP=[]
for name in xrpusdtColumNames:
  renamedNamesXRP.append(name+"_XRPUSDT")
      
renamedNamesXRP=dict(zip(xrpusdtColumNames, renamedNamesXRP))
xrpusdt=xrpusdt.rename(index=str, columns=renamedNamesXRP)


main_df= pd.concat([main_df.reset_index(drop=True),ltcusdt.reset_index(drop=True)], axis=1)
main_df= pd.concat([main_df.reset_index(drop=True),xrpusdt.reset_index(drop=True)], axis=1)
main_df.index = [dt.datetime.fromtimestamp(x/1000.0) for x in main_df.open_time]
main_df=main_df.drop(columns=['open_time','open_time_LTCUSDT','open_time_XRPUSDT'])


main_df['future'] = main_df[f'close_price_{RATIO_TO_PREDICT}'].shift(-FUTURE_PERIOD_PREDICT)
main_df['target'] = list(map(classify, main_df[f'close_price_{RATIO_TO_PREDICT}'], main_df['future']))



1542667601353
1542727601353
1542787601353
1542847601353
1542907601353
1542967601353
5760


In [0]:
#Separate Validation data -- Last VALIDATION_PERCENTAGE of the data

times = sorted(main_df.index.values)
last_Xpct = times[-int(VALIDATION_PERCENTAGE*len(times))]

validation_data = main_df[(main_df.index >= last_Xpct)]
main_df= main_df[(main_df.index < last_Xpct)]

print(len(main_df))
print(len(validation_data))
print(main_df.head())
print(validation_data.head())

In [0]:
preprocess(main_df)
#train_x,train_y = preprocess_df(main_df)
#validation_x, validation_y = preprocess_df(validation_data)
